<a href="https://colab.research.google.com/github/anfalaugust/CSC496_Models/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.listdir("/content/drive/MyDrive/")

!!unzip "/content/drive/MyDrive/CSC496/Diseases of date palm leaves dataset.zip" -d "/content/dataset"

import tensorflow as tf
os.listdir("/content/dataset")

dataset_path = "/content/dataset/Diseases of date palm leaves dataset/Infected Date Palm Leaves Dataset/Processed"

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    image_size=(224, 224),
    batch_size=32
)

print("Classes:", train_ds.class_names)

Mounted at /content/drive
Found 3089 files belonging to 9 classes.
Classes: ['1. Potassium Deficiency', '2. Manganese Deficiency', '3. Magnesium Deficiency', '4. Black Scorch', '5. Leaf Spots', '6. Fusarium Wilt', '7. Rachis Blight', '8. Parlatoria Blanchardi', '9. Healthy sample']


In [5]:
# ==============================
# DenseNet121: Load Data -> Train -> Evaluate (One Cell)
# ==============================

import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# ==============================
# 1) Paths & Parameters
# ==============================

dataset_path = "/content/dataset/Diseases of date palm leaves dataset/Infected Date Palm Leaves Dataset/Processed"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42
EPOCHS = 15

# ==============================
# 2) Load Dataset (80/20 Split)
# ==============================

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)

print("Classes:", class_names)
print("Number of Classes:", num_classes)

# ==============================
# 3) Optimize Pipeline
# ==============================

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# ==============================
# 4) Build Model
# ==============================

def build_model(num_classes):

    base_model = tf.keras.applications.DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(224,224,3)
    )

    base_model.trainable = False  # Freeze backbone

    model = tf.keras.Sequential([
        layers.Rescaling(1./255),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

model = build_model(num_classes)

model.summary()

# ==============================
# 5) Compile
# ==============================

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ==============================
# 6) Train
# ==============================

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

# ==============================
# 7) Evaluate
# ==============================

val_loss, val_acc = model.evaluate(val_ds)

print("✅ Validation Accuracy:", round(val_acc * 100, 2), "%")
print("✅ Validation Loss:", val_loss)

Found 3089 files belonging to 9 classes.
Using 2472 files for training.
Found 3089 files belonging to 9 classes.
Using 617 files for validation.
Classes: ['1. Potassium Deficiency', '2. Manganese Deficiency', '3. Magnesium Deficiency', '4. Black Scorch', '5. Leaf Spots', '6. Fusarium Wilt', '7. Rachis Blight', '8. Parlatoria Blanchardi', '9. Healthy sample']
Number of Classes: 9
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet121 (Functional)        │ (None, 7, 7, 1024)     │     7,037,504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ ?                      │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,037,504 (26.85 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 7,037,504 (26.85 MB)

Epoch 1/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 81s 578ms/step - accuracy: 0.1922 - loss: 2.3954 - val_accuracy: 0.4635 - val_loss: 1.7117
Epoch 2/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 91ms/step - accuracy: 0.4690 - loss: 1.5886 - val_accuracy: 0.6045 - val_loss: 1.3694
Epoch 3/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 89ms/step - accuracy: 0.5782 - loss: 1.2667 - val_accuracy: 0.6726 - val_loss: 1.1443
Epoch 4/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 93ms/step - accuracy: 0.6304 - loss: 1.1004 - val_accuracy: 0.7245 - val_loss: 0.9827
Epoch 5/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 94ms/step - accuracy: 0.6729 - loss: 0.9282 - val_accuracy: 0.7488 - val_loss: 0.8631
Epoch 6/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 98ms/step - accuracy: 0.7170 - loss: 0.8288 - val_accuracy: 0.7682 - val_loss: 0.7700
Epoch 7/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 99ms/step - accuracy: 0.7480 - loss: 0.7885 - val_accuracy: 0.7958 - val_loss: 0.6976
Epoch 8/15
78/78 ━━━━━━━━━━━━━━━━━━━━ 8s 101ms/step - accuracy: 0.7550 - loss: 0.7015 - val_accuracy: 0.8120 

In [ ]:
# ==============================
# YOLOv8 Classification: Load -> Train -> Evaluate
# ==============================

!pip install ultralytics -q

import os
import shutil
import random
from ultralytics import YOLO

IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 10
SEED = 42

# 1️⃣ Prepare YOLO dataset structure (train/val split 80/20)
yolo_dataset_path = "/content/dataset/Diseases of date palm leaves dataset/Infected Date Palm Leaves Dataset/Processed"

if not os.path.exists(yolo_dataset_path):
    os.makedirs(yolo_dataset_path, exist_ok=True)

    for split in ["train", "val"]:
        os.makedirs(os.path.join(yolo_dataset_path, split), exist_ok=True)

    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path):
            continue

        images = os.listdir(class_path)
        random.seed(SEED)
        random.shuffle(images)

        split_index = int(0.8 * len(images))
        train_images = images[:split_index]
        val_images = images[split_index:]

        for split, image_list in zip(["train", "val"], [train_images, val_images]):
            split_class_dir = os.path.join(yolo_dataset_path, split, class_name)
            os.makedirs(split_class_dir, exist_ok=True)

            for img in image_list:
                shutil.copy(
                    os.path.join(class_path, img),
                    os.path.join(split_class_dir, img)
                )

print("Dataset ready for YOLO!")

# 2️⃣ Load YOLOv8 classification model (nano version)
model = YOLO("yolov8n-cls.pt")

# 3️⃣ Train
results = model.train(
    data=yolo_dataset_path,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE
)

# 4️⃣ Evaluate
metrics = model.val()

print("✅ Top-1 Validation Accuracy:", metrics.top1)
print("✅ Top-5 Validation Accuracy:", metrics.top5)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Dataset ready for YOLO!
Ultralytics 8.4.16 🚀 Python-3.12.12 torch-2.10.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/Diseases of date palm leaves dataset/Infected Date Palm Leaves Dataset/Processed, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, ep

In [ ]:
# ==============================
# EfficientNetB0: Load Data -> Train -> Evaluate (One Cell)
# ==============================

import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# ==============================
# 1) Paths & Parameters
# ==============================

dataset_path = "/content/dataset/Diseases of date palm leaves dataset/Infected Date Palm Leaves Dataset/Processed"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 123
EPOCHS = 10

# ==============================
# 2) Load Dataset (80/20 Split)
# ==============================

train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_path,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
num_classes = len(class_names)

print("Classes:", class_names)
print("Number of Classes:", num_classes)

# ==============================
# 3) Optimize Pipeline
# ==============================

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# ==============================
# 4) Build Model
# ==============================

def build_model(num_classes):

    base_model = tf.keras.applications.EfficientNetB0(
        weights='imagenet',
        include_top=False,
        input_shape=(224,224,3)
    )

    base_model.trainable = False  # Freeze backbone

    model = tf.keras.Sequential([
        # ❗ DO NOT add Rescaling for EfficientNet
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.BatchNormalization(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(num_classes, activation='softmax')
    ])

    return model

model = build_model(num_classes)

model.summary()

# ==============================
# 5) Compile
# ==============================

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# ==============================
# 6) Train
# ==============================

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

# ==============================
# 7) Evaluate
# ==============================

val_loss, val_acc = model.evaluate(val_ds)

print("✅ Validation Accuracy:", val_acc )
print("✅ Validation Loss:", val_loss)

Found 3089 files belonging to 9 classes.
Using 2472 files for training.
Found 3089 files belonging to 9 classes.
Using 617 files for validation.
Classes: ['1. Potassium Deficiency', '2. Manganese Deficiency', '3. Magnesium Deficiency', '4. Black Scorch', '5. Leaf Spots', '6. Fusarium Wilt', '7. Rachis Blight', '8. Parlatoria Blanchardi', '9. Healthy sample']
Number of Classes: 9


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1280)           │         5,120 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 9)              │         1,161 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,219,820 (16.10 MB)

 Trainable params: 167,689 (655.04 KB)

 Non-trainable params: 4,052,131 (15.46 MB)

Epoch 1/10
 8/78 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.1343 - loss: 3.1922